In [3]:
from __future__ import unicode_literals
from __future__ import print_function
import glob, os, sys,re,json
import numpy as np
import spacy
from sklearn.utils import shuffle

reload(sys)
sys.setdefaultencoding('utf8')

nlp = spacy.load('en')

import sys,json,gzip,random,re,math
from collections import Counter
import numpy as np
from sklearn.cross_validation import KFold
from sklearn import linear_model
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV

from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import dynamic_rnn

import librosa

<h2> Load in files from the Liu et al. 2017 dataset and read vocab </h2>

In [2]:
def get_liu_transcript_paths():
    pathname="/data/corpora/ted/transcripts_clean/"
    files = [pathname + f for f in os.listdir(pathname)]
    talk_names_path = "talk_names.txt"
    talk_names = open(talk_names_path).read().replace('.txt','.html').split('\n')
    talk_names = ['isabel_allende_how_to_live_passionately_no_matter_your_age.html' if t == 'isabelle_allende_how_to_live_passionately_no_matter_your_age.html' else t for t in talk_names]
    talk_names = [t for t in talk_names if t not in ['', 'test.html', '\r']]
    transcripts = [pathname + t for t in talk_names]
    return(transcripts)

#transcripts = get_liu_transcript_paths()
#len(transcripts)

def get_transcript_paths():
    pathname="/data/corpora/ted/transcripts_clean/"
    files = [pathname + f for f in os.listdir(pathname)]
    return files

transcripts = get_transcript_paths()
len(transcripts)

2483

In [498]:
len(good_transcript_paths)

1068

In [3]:
def readLIWC(filename):
    liwc_vocab={}
    regex_liwc={}
    liwc={}
    
    # Liu excludes "affective or emotional processes"
    invalid={}
    invalid["31"]=1
    invalid["32"]=1
    invalid["33"]=1
    invalid["34"]=1
    invalid["35"]=1

    file=open(filename)
    for i in range(75):
        line=file.readline()
        cols=re.split("\s+", line.rstrip())
        if line.rstrip() != "%":
            idd="%s" % cols[0]
            label=cols[1]
            liwc_vocab[idd]=label
    for line in file:
        cols=line.rstrip().split("\t")
        term=cols[0]
        valid=[]
        for x in cols[1:]:
            if x not in invalid:
                valid.append(x)
        cats=["LIWC_%s" % liwc_vocab[x] for x in valid]

        if term.endswith("*"):
            pref=term[0:2]
            if pref not in regex_liwc:
                regex_liwc[pref]={}
            regex_liwc[pref][term]=cats
        else:
            liwc[term]=cats
            
    return (liwc_vocab, regex_liwc, liwc)

# get all flat LIWC categories for a word
def getLIWC(word):
    vals=[]
    if word in liwc:
        vals.extend(liwc[word])
    if len(word) > 1:
        pref=word[0:2]
        if pref in regex_liwc:
            cands=regex_liwc[pref]
            for cand in cands:
                if re.match(cand, word) != None:
                    vals.extend(regex_liwc[pref][cand])
    return vals

def get_words(transcript_path):
    words = []
    text = open(transcript_path).read()
    lines = text.split("\n")
    lines = [l for l in lines if l != '']
    paragraphs = [line.split("\t")[1] for line in lines]
    paragraphs = [p.replace("(Laughter)","").replace("(Applause)","") for p in paragraphs]
    for p in paragraphs:
        words += p.split(' ')
    return words
    

def get_vocab(transcript_path_list):
    vocab = {}
    
    words = []
    for t in transcript_path_list:
        words += get_words(t)
        #words += t.get_words()
        
    fid = 1
    
    for key in liwc_vocab:
        feat="LIWC_%s" % (liwc_vocab[key])
        vocab[feat]=fid
        fid+=1
        
    counts=Counter()
    
    for word in words:
        counts[word.lower()]+=1
    
    for word in counts:
        count=counts[word]
        #if count >= 500:
        if count >= 500000:
            vocab[word.lower()]=fid
            fid+=1
            
    return vocab

In [4]:
liwc_path = '/data/corpora/LIWC/LIWC.txt'
liwc_vocab, regex_liwc, liwc = readLIWC(liwc_path)
vocab = get_vocab(transcripts)

In [499]:
good_transcript_paths[0]

u'/data/corpora/ted/transcripts_clean/linus_torvalds_the_mind_behind_linux.html'

In [5]:
alignments_dir = '/data/corpora/ted/forced_alignments/'
alignments_files = os.listdir(alignments_dir)

In [7]:
matched = []
for t in transcripts:
    matcher = t.split("/")[-1].replace(".html", ".json")
    if matcher in alignments_files:
        matched.append(t)

In [8]:
len(matched)

866

In [74]:
def cstm(s):
    n_mins = int(s) / 60
    n_secs = s - 60*n_mins
    return ("%i:%02.1f") % (n_mins, n_secs)

In [14]:
def get_pkl_path(filename):
    return '/data/jrgillick/applause/transcripts/' + filename.split('transcripts_clean/')[1].split('.html')[0] + '.pkl'

In [1317]:
get_pkl_path(t.filename)

u'/data/jrgillick/applause/transcripts/abigail_washburn_building_us_china_relations_by_banjo.pkl'

In [9]:
import pickle

In [1324]:
for transcript in matched[0:100]:
    pkl_path = get_pkl_path(transcript)
    if not os.path.exists(pkl_path):
        t = Transcript(transcript)
        t.compute_sentence_audio_features()    
        with open(pkl_path,'wb') as f:
            pickle.dump(t, f)

In [1287]:
import time
t0 = time.time()
t.compute_sentence_audio_features()
time_elapsed = time.time() - t0

In [1288]:
time_elapsed

231.1519889831543

In [1294]:
t.sentence_audio_features[3]

[0.057394475,
 0.0018195921,
 0.12844008,
 0.12662049,
 0.032966193,
 3.1899773242630385,
 3.1348185217304767,
 33.251755,
 105.0,
 -1.0,
 106.0,
 37.384201,
 0.5023622047244094]

In [1051]:
t.sentences[0][0]

u'Most  of  us  think  of  motion  as  a  very  visual  thing . '

In [1078]:
i=59
zip([s[0] for s in t.sentences], [(cstm(at[0]),cstm(at[1])) for at in t.get_alignment_times()])[i]

(u'AD :  And  there  are  other  ways  that  we  can  push  these  limits  as  well . ',
 (u'9:14.5', u'9:25.1'))

In [1052]:
[(i,s[0]) for i, s in enumerate(t.sentences) if s[2] == 1]

[(48, u'Little  lamb !'),
 (58,
  u'( Audio )  Mary  had  a  little  lamb  whose  fleece  was  white  as  snow ,  and  everywhere  that  Mary  went ,  that  lamb  was  sure  to  go .'),
 (86,
  u"And  so  what  you  see  here  is  a  simulation  of  how  this  object  would  respond  to  new  forces  that  we 've  never  seen  before ,  and  we  created  it  from  just  five  seconds  of  regular  video ."),
 (90,
  u"So  for  example ,  here 's  a  video  that  I  captured  of  a  bush  outside  of  my  apartment ,  and  I  did n't  do  anything  to  this  bush ,  but  by  capturing  a  minute - long  video ,  a  gentle  breeze  caused  enough  vibrations  that  we  could  learn  enough  about  this  bush  to  create  this  simulation . "),
 (94,
  u'So  here  are  the  amazing  people  who  worked  with  me  on  these  projects . ')]

In [1041]:
all_transcripts = [t for t in all_transcripts if t.filename in matched]

In [1083]:
len(all_transcripts)

866

In [1084]:
t.get_alignment_times()[0]

(13.389999, 16.759999999999998)

In [1295]:
t.filename

u'/data/corpora/ted/transcripts_clean/abigail_washburn_building_us_china_relations_by_banjo.html'

In [6]:
import pyAudioAnalysis
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction

def get_pitch_tracking(audio, sr):
    r = str(np.random.randint(999999))
    librosa.output.write_wav('temp_%s.wav' % (r) ,audio,sr)
    reaper_cmd = "~/REAPER/build/reaper -i temp_%s.wav -f temp_%s.f0 -p temp_%s.pm -a" % (r,r,r)
    os.system(reaper_cmd)
    txt = open('temp_%s.f0' %(r)).read()
    txt = txt.split('\n')[7:][0:-1]
    split_lines = [l.split(' ') for l in txt]
    cleanup_cmd = "rm temp_%s.wav temp_%s.f0 temp_%s.pm" %(r,r,r)
    os.system(cleanup_cmd)
    return split_lines

def compute_audio_energy_features(audio):
    rmse = librosa.feature.rmse(audio,frame_length=1024)
    mean_energy = np.mean(rmse)
    min_energy = np.min(rmse)
    max_energy = np.max(rmse)
    range_energy = max_energy - min_energy
    std_energy = np.std(rmse)
    return [mean_energy, min_energy, max_energy, range_energy, std_energy]

def compute_audio_temporal_features(s,audio,sr):
    duration = len(audio) / float(sr)
    tempo = len(s) / duration
    return [duration, tempo]
    
def compute_audio_pitch_features(audio,sr):
    pitch_list = np.array([p[2] for p in get_pitch_tracking(audio,sr)]).astype(np.float32)
    mean_pitch = np.mean(pitch_list)
    max_pitch = np.max(pitch_list)
    min_pitch = np.min(pitch_list)
    range_pitch = max_pitch - min_pitch
    std_pitch = np.std(pitch_list)
    internal_silence = np.sum(pitch_list == -1) / float(len(pitch_list))
    return [mean_pitch,max_pitch,min_pitch,range_pitch,std_pitch,internal_silence]
    
def compute_audio_features(s,audio,sr):
    return compute_audio_energy_features(audio) + compute_audio_temporal_features(s,audio,sr) + compute_audio_pitch_features(audio,sr)

<h2> Create "Transcript" object to compute features from text files </h2>

In [4]:
class Transcript:
    def __init__(self, filename):
        self.filename = filename
        self.sequences = self.get_sequences()
        self.sentences = self.get_sentence_list()
        self.full_text = self.get_sentence_text()
        #self.alignment = self.get_alignments_file()
        #self.alignment_times = self.get_alignment_times()
        
    def get_text(self):
        return open(self.filename).read()
    
    def get_split_lines(self):
        text = self.get_text()
        lines = text.split("\n")
        lines = [l for l in lines if l != '']
        return [line.split("\t") for line in lines]
    
    def get_paragraphs(self):
        split_lines = self.get_split_lines()
        return [l[1] for l in split_lines]
        
    def get_timestamps(self):
        split_lines = self.get_split_lines()
        return [l[0] for l in split_lines]
        
    def get_sentences(self, paragraph):
        #par = unicode(paragraph).encode("utf-8")
        paragraph = paragraph.replace('(Laughter)','')
        sentences = []
        doc = nlp(paragraph)
        for sentence in doc.sents:
            words=[]
            for word in sentence:
                if re.search("\S", word.string) != None:
                    words.append(word.string)
            text=' '.join(words)
            if re.match("^\(.*?\)$", text) != None or re.search("\w", text) == None:
                continue
            sentences.append(text)
        return sentences
    
    def get_words(self):
        words = []
        paragraphs = [p.replace("(Laughter)","").replace("(Applause)","") for p in self.get_paragraphs()]
        for p in paragraphs:
            words += p.split(' ')
        return words
    
    def get_sentences_with_applause_interspersed(self, paragraph):
        paragraph = paragraph.replace('(Laughter)','')
        lines = paragraph.split("(Applause)")
        last_line = lines[-1]; lines = lines[0:-1]  # Don't add applause after last line
        sentences = []
        for line in lines:
            sentences += self.get_sentences(line)
            sentences.append("(Applause)")
        sentences += self.get_sentences(last_line)
        return sentences
      
    def get_sequences(self):
        sequences = []
        split_lines = self.get_split_lines()
        for line in split_lines:
            if "(Applause)" in line[1] and line[1] != "(Applause)":
                timestamp = line[0]; paragraph = line[1]
                sentences = self.get_sentences_with_applause_interspersed(paragraph)
                sequences.append((timestamp, paragraph, sentences))
                #special_cases.append(line[1])
            elif line[1] == "(Applause)":
                timestamp = line[0]; paragraph = line[1]; sentences = ["(Applause)"]
                sequences.append((timestamp, paragraph, sentences))
            else:
                timestamp = line[0]; paragraph = line[1]; sentences = self.get_sentences(paragraph)
                sequences.append((timestamp, paragraph, sentences))
        return sequences
    
    def get_sentence_list(self):
        all_sentences = []
        seqs = self.sequences[0:-1] #exclude last paragraph, which may be final applause
        for sequence_index, sequence in enumerate(seqs): 
            sentences = sequence[2]
            for sentence_index, s in enumerate(sentences):
                if s != "(Applause)":
                    features = np.concatenate((self.get_dense_liwc_features(s),self.get_word_vec_features(s)))
                    # If this is not the last sentence in the seq, check if next sentence in this sequence is applause
                    if sentence_index < len(sentences)-1 and sentences[sentence_index+1] == "(Applause)":
                        applause_follows = 1
                    # If this is the last sentence of this sequence, check first sentence of next sequence
                    elif sentence_index == len(sentences) -1 and sequence_index < len(seqs) - 1:
                        next_sentences = seqs[sequence_index+1][2]
                        if len(next_sentences) > 0 and next_sentences[0] == "(Applause)":
                            applause_follows = 1
                        else:
                            applause_follows = 0    
                    else:
                        applause_follows = 0
                    all_sentences.append((s,features,applause_follows))
        return all_sentences
    
    def get_sentence_text(self):
        return " ".join([s[0].replace(" \'","\'") for s in self.sentences])
    
    def get_indices_of_sentence_in_full_txt(self,s):
        s = s.replace(" \'","\'")
        start = self.full_text.index(s)
        end = start + len(s)
        return (start, end)
    
    def get_last_detected_end_time(self, alignment, index):
        while index >= 0:
            if index < len(alignment['words']):
                word = alignment['words'][index]
                if 'end' in word.keys():
                    return word['end']
            index -= 1
    
    def get_last_detected_start_time(self, alignment, index):
        if index >= len(alignment['words']):
            index = len(alignment['words']) - 1
        word = alignment['words'][index]
        if 'start' in word.keys():
            return word['start']
        try:
            next_word = alignment['words'][index+1]
            if 'start' in next_word.keys():
                return next_word['start'] - 0.1
        except:
            last_word = alignment['words'][index-1]
            if 'start' in last_word.keys():
                return last_word['end'] - 0.1
        
        while index >= 0:
            word = alignment['words'][index]
            if 'start' in word.keys():
                return word['start']
            index -= 1
    
    def get_sentence_words(self, s):
        s = s.replace(" \'","\'").replace("\u2014","").replace("[","").replace("]","").replace("@","").replace("  "," ")
        s_words = re.split("[, \-!?:.&\/\"]+",s)
        s_words = [s for s in s_words if re.search("\w", s) is not None]
        s_words = [s[0:-1] if s[-1] in ["\'","\u2014","%"] else s for s in s_words ]
        return s_words
    
    def get_alignment_times(self):
        sentences = [s[0] for s in self.sentences]
        alignment = self.alignment
        alignment_index = 0
        alignment_times = []
        for idx, s in enumerate(sentences):
            s_words = self.get_sentence_words(s)
            alignment_words = alignment['words'][alignment_index:alignment_index+len(s_words)]
            original_alignment_index = alignment_index
            n_tries = 0; jump_amounts = [1,-1,2,-2,3]
            if idx > 0 and idx < len(sentences) and len(s_words) > 0 and len(alignment_words) > 0:
                while n_tries < 5 and s_words[0].lower() != alignment_words[0]['word'].lower():
                    alignment_index = original_alignment_index + jump_amounts[n_tries]
                    alignment_words = alignment['words'][alignment_index:alignment_index+len(s_words)]
                    n_tries += 1
            alignment_start = self.get_last_detected_start_time(alignment, alignment_index) #alignment['words'][alignment_index]['start']
            alignment_end = self.get_last_detected_end_time(alignment, alignment_index + len(s_words) -1)#alignment['words'][alignment_index+len(s_words)]['end']
            alignment_times.append((alignment_start,alignment_end))
            alignment_index += len(s_words)
        alignment_times = self.smooth_alignment_times(alignment_times)
        return alignment_times
    
    def smooth_alignment_times(self, alignment_times):
        for i in range(len(alignment_times)):
            start, end = alignment_times[i]
            if start is None and end is not None:
                alignment_times[i] = (end-0.1, end)
            if start is not None and end is None:
                alignment_times[i] = (start, start+0.1)
            if start is None and end is None:
                counter = 1
                next_start = None
                while next_start is None:
                    next_start = alignment_times[i+counter][0]
                    counter+= 1
                alignment_times[i] = (next_start-0.2, next_start-0.1)
            if alignment_times[i][1] - alignment_times[i][0] < 0.1:
                alignment_times[i] = (alignment_times[i][0], alignment_times[i][0]+0.1)
        return alignment_times
    
    def get_sentence_audio_features(self):
        audio_path = self.filename.replace("transcripts_clean","audio").replace(".html",".mp3")
        y,sr = librosa.load(audio_path)
        all_audio_features = []
        for i in range(len(self.sentences)):
            s = self.get_sentence_words(self.sentences[i][0])
            start, end = self.alignment_times[i]
            audio = y[int(sr*start):int(sr*end)]
            audio_feats = compute_audio_features(s,audio,sr)
            all_audio_features.append(audio_feats)
        return all_audio_features
    
    def compute_sentence_audio_features(self):
        self.sentence_audio_features = self.get_sentence_audio_features()          
    
    def get_word_vector_list(self, sentence):
        vecs = []
        for word in nlp(sentence):
            if re.search("\S", word.string) != None: vecs.append(word.vector)
        return np.array(vecs)
    
    def compute_word_features(self):
        self.word_features = []
        for index, sentence in enumerate(self.sentences):
            s = sentence[0]
            word_vecs = self.get_word_vector_list(s)
            self.word_features.append(word_vecs)# self.sentences[index] = (sentence[0], sentence[1], sentence[2], word_vecs)
    
    def get_alignments_file(self):
        sentences = [s[0].replace(" '","'") for s in self.sentences]
        alignment_file = self.filename.replace('transcripts_clean','forced_alignments').replace('.html','.json')
        return json.loads(open(alignment_file).read())
    
    def get_words_in_sentence(self, sentence):
        return [word for word in nlp(sentence) if re.search("\S", word.string) != None]

    def count_words_in_sentence(self, sentence):
        return len(self.get_words_in_sentence(sentence))
    
    def compute_word_count_list(self):
        self.word_count_list = []
        for sentence in self.sentences:
            self.word_count_list.append(self.count_words_in_sentence(sentence[0]))
            
    def count_preceding_words(self, index):
        return np.sum (self.word_count_list[0:index+1])
    
    def count_applause_instances(self):
        c = 0
        sentence_list = self.sentences
        for s in sentence_list:#[0:-1]:
            if s[2] == 1:
                c += 1
        return c
    
    def get_applause_yes_sentences(self):
        return [s for s in self.sentences if s[2] == 1]
    
    def get_applause_no_sentences(self):
        no_sentences = [s for s in self.sentences if s[2] == 0]
        shuffle(no_sentences)
        return no_sentences[0:len(self.get_applause_yes_sentences())]
    
    def get_sparse_liwc_features(self, sentence):
        counts = {}
        text = sentence.lower().split(' ')
        for word in text:
            cats=getLIWC(word)
            for cat in cats:
                if vocab[cat] in counts:
                    counts[vocab[cat]] += 1  # = 1
                else:
                    counts[vocab[cat]] = 1.
            if word in vocab:
                if vocab[word] in counts:
                    counts[vocab[word]] += 1  # = 1
                else:
                    counts[vocab[word]] = 1
        return counts
    
    def get_dense_liwc_features(self,sentence):
        a = np.zeros(len(vocab))
        sparse_feats = self.get_sparse_liwc_features(sentence)
        for k in sparse_feats.keys():
            a[k-1] = float(sparse_feats[k]) / len(sentence)
        return a
        
    def get_word_vec_features(self,sentence):
        return nlp(sentence).vector

In [8]:
all_transcripts = [Transcript(t) for t in transcripts]

In [11]:
good_transcripts = []
for t in all_transcripts:
    if t.count_applause_instances() > 0:
        good_transcripts.append(t)

In [21]:
len(good_transcripts)

1068

In [26]:
good_transcript_paths = [t.filename for t in good_transcripts]

In [27]:
import pickle

In [25]:
with open('good_transcript_paths.txt','wb') as f:
    for p in good_transcript_paths:
        f.write(p + '\n')

In [28]:
with open('good_transcripts.pkl','wb') as f:
    pickle.dump(good_transcripts, f)

In [6]:
import pickle, numpy as np
with open('good_transcripts.pkl','rb') as f:
    good_transcripts = pickle.load(f)

In [7]:
len(good_transcripts)

1068

In [27]:
from tqdm import tqdm

In [30]:
ted_text_and_labels = []
for t in tqdm(good_transcripts):
    text_and_labels = [(str(t.sentences[i][0]),t.sentences[i][2]) for i in range(len(t.sentences))]
    ted_text_and_labels.append(text_and_labels)
    
with open('/data/jrgillick/ted_text_and_labels.pkl','wb') as f:
    pickle.dump(ted_text_and_labels,f)

In [14]:
alignments_dir = '/data/corpora/ted/forced_alignments/'
alignments_files = os.listdir(alignments_dir)

matched = []
for t in good_transcripts:
    matcher = t.filename.split("/")[-1].replace(".html", ".json")
    if matcher in alignments_files:
        matched.append(t)

In [37]:
len(matched)

786

In [41]:
all_transcripts = []
p = '/data/jrgillick/applause/transcripts/'
paths = [p + path for path in os.listdir(p)]
for path in paths:
    with open(path, "rb") as f:
        t = pickle.load(f)
        all_transcripts.append(t)

In [39]:
paths[0]

u'evan_grant_cymatics.pkl'

In [42]:
len(all_transcripts)

866

In [8]:
# Store objects so we don't have to recompute features 

import pickle

def save_transcripts(filename="transripts.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(all_transcripts, f)

def load_transcripts(filename="transcripts.pkl"):
    with open(filename, 'rb') as f:
        return pickle.load(f)

transcripts_path = 'transcripts.pkl'

if os.path.exists(transcripts_path):
    all_transcripts = load_transcripts(transcripts_path)
    
else:
    all_transcripts = [Transcript(f) for f in transcripts]
    save_transcripts(transcripts_path)
    
#all_transcripts = shuffle(all_transcripts)

In [9]:
t = all_transcripts[0]

In [10]:
t.get_alignment_times()

AttributeError: Transcript instance has no attribute 'alignment'

In [15]:
all_transcripts = []
p = '/data/jrgillick/applause/transcripts/'
files = [p+f for f in os.listdir(p)]
for fl in files:
    with open(fl) as f:
        all_transcripts.append(pickle.load(f))

In [200]:
t = all_transcripts[10]

In [201]:
sentences = t.sentences

In [202]:
[(i,s[0],s[2]) for i,s in enumerate(sentences) if s[2]==1]

[(168,
  u"( Video )  Johnny  Cash :  \u266b  There  ai n't  no  grave  \u266b  \u266b  can  hold  my  body  down  \u266b  \u266b  There  ai n't  no  grave  \u266b  \u266b  can  hold  body  down  \u266b  \u266b  When  I  hear  the  trumpet  sound  \u266b  \u266b  I 'm  going  to  ride  right  out  of  the  ground  \u266b  \u266b  Ai n't  no  grave  \u266b  \u266b  can  hold  my  body  ...  \u266b",
  1)]

In [188]:
#[(i,s[0],s[2]) for i,s in enumerate(sentences) if i>105 and i<112]

In [203]:
t.filename

u'/data/corpora/ted/transcripts_clean/aaron_koblin.html'

In [204]:
i = 68
t.alignment_times[i]

(296.98, 297.75)

In [197]:
#for t in all_transcripts:
#    t.compute_alignments()
#    t.compute_word_features()
#    t.compute_word_count_list()

In [198]:
cstm(t.alignment_times[i][0])

u'2:53.7'

In [199]:
cstm(t.alignment_times[i][1])

u'3:6.3'

In [26]:
t = all_transcripts[0]

In [30]:
np.array(t.sentence_audio_features).shape

(40, 13)

In [44]:
def get_applause_yes_features(t):
    features = []
    sentences = t.sentences
    for i in get_applause_indices(t):
        features.append(get_feature(t,i))
    return features

def get_feature(t,i):
    word_features = list(t.sentences[i][1])
    audio_features = list(t.sentence_audio_features[i])
    return word_features+audio_features

def get_applause_indices(t):
    return [i for i,s in enumerate(t.sentences) if s[2] == 1]

def get_non_applause_indices(t):
    return [i for i,s in enumerate(t.sentences) if s[2] != 1]

def get_applause_no_features(t):
    non_applause_indices = get_non_applause_indices(t)
    shuffle(non_applause_indices)
    features = []
    for i in non_applause_indices[0:len(get_applause_indices(t))]:
        features.append(get_feature(t,i))
    return features

In [45]:
np.array(get_applause_no_features(t) + get_applause_no_features(t)).shape

(4, 386)

<h2> Run Logistic Regression Models </h2>

In [46]:
from sklearn.cross_validation import KFold
from sklearn import linear_model
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support

def get_balanced_applause_features(all_transcripts):
    applause_yes = []
    applause_no = []
    for t in all_transcripts:
        for a in t.get_applause_yes_sentences():
            applause_yes.append(a[1])
        for a in t.get_applause_no_sentences():
            applause_no.append(a[1])
    applause_yes = np.array(applause_yes)
    applause_no = np.array(applause_no)
    return applause_yes, applause_no

def get_unbalanced_applause_features(all_transcripts):
    applause_yes = []
    applause_no = []
    for t in all_transcripts:
        for a in t.get_applause_yes_sentences():
            applause_yes.append(a[1])
        for a in t.sentences:
            if a[2] == 0:
                applause_no.append(a[1])
    applause_yes = np.array(applause_yes)
    applause_no = np.array(applause_no)
    return applause_yes, applause_no

def get_balanced_train_data_and_labels(all_transcripts):
    applause_yes, applause_no = get_balanced_applause_features(all_transcripts)
    X = np.vstack([applause_yes,applause_no])
    Y = np.concatenate([np.ones(len(applause_yes)),np.zeros(len(applause_no))])
    return X, Y

def get_unbalanced_train_data_and_labels(all_transcripts):
    applause_yes, applause_no = get_unbalanced_applause_features(all_transcripts)
    X = np.vstack([applause_yes,applause_no])
    Y = np.concatenate([np.ones(len(applause_yes)),np.zeros(len(applause_no))])
    return X, Y

In [47]:
def get_balanced_applause_features_with_audio(all_transcripts):
    applause_yes = []
    applause_no = []
    for t in all_transcripts:
        applause_yes += get_applause_yes_features(t)
        applause_no += get_applause_no_features(t)
    return np.array(applause_yes), np.array(applause_no)

def get_balanced_train_data_and_labels_with_audio(all_transcripts):
    applause_yes, applause_no = get_balanced_applause_features_with_audio(all_transcripts)
    X = np.vstack([applause_yes,applause_no])
    Y = np.concatenate([np.ones(len(applause_yes)),np.zeros(len(applause_no))])
    return X, Y

In [48]:
get_balanced_applause_features_with_audio(all_transcripts)[1].shape

(1910, 386)

<h3> Logistic Regression on balanced data following Liu et al. 2017</h3>

In [70]:
total_true = []
total_pred = []

X,Y = get_balanced_train_data_and_labels_with_audio(all_transcripts)
X,Y = shuffle(X,Y)
kf = KFold(len(Y), n_folds=10)

for train_index, test_index in kf:
    #X_train, X_test = X[train_index][:,73:], X[test_index][:,73:]
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    logreg=linear_model.LogisticRegression(penalty='l1',C=4)
    clf = logreg
    clf = GridSearchCV(logreg, {'C':(0.001, .01, .1, 1, 3, 4, 5)}, cv=3)
    #clf = GridSearchCV(logreg, {'C':(3.5,4,4.5)}, cv=3)
    clf.fit(X_train, y_train)
    
    #print(clf.best_params_, len(y_train))
    y_true, y_pred = y_test, clf.predict(X_test)
    for i in range(len(y_true)):
        
        total_true.append(y_true[i])
        total_pred.append(y_pred[i])

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.734 +/- 0.014 (2802/3820)'

In [71]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

'Precision: 0.757 | Recall: 0.688 | F1: 0.721'

In [72]:
clf.best_params_

{u'C': 4}

<h3> Run cross validation again with clean split on full talks rather than sentences </h3>

In [76]:
total_true = []
total_pred = []

kf = KFold(len(all_transcripts), n_folds=10)

for train_index, test_index in kf:
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)
    X_test, y_test = get_balanced_train_data_and_labels_with_audio(t_test)

    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)

    #X_train, X_test = X_train[:,0:73], X_test[:,0:73]
    
    logreg=linear_model.LogisticRegression(penalty='l1',C=3)
    clf = logreg

    clf = GridSearchCV(logreg, {'C':(3,3.5,4,4.5)}, cv=3)
    clf.fit(X_train, y_train)
    
    y_true, y_pred = y_test, clf.predict(X_test)
    for i in range(len(y_true)):
        
        total_true.append(y_true[i])
        total_pred.append(y_pred[i])

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.722 +/- 0.014 (2759/3820)'

In [77]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

'Precision: 0.746 | Recall: 0.674 | F1: 0.708'

In [75]:
clf.best_params_

{u'C': 3.5}

<h3> Train on balanced subset, test on full talks </h3>

In [596]:
total_true = []
total_pred = []

for train_index, test_index in kf:
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    X_train, y_train = get_balanced_train_data_and_labels(t_train)
    X_test, y_test = get_unbalanced_train_data_and_labels(t_test)

    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)

    X_train, X_test = X_train[:,73:], X_test[:,73:]
    
    logreg=linear_model.LogisticRegression(penalty='l1')
    clf = logreg

    clf = GridSearchCV(logreg, {'C':(3,3.5,4,4.5)}, cv=3)
    clf.fit(X_train, y_train)
    
    y_true, y_pred = y_test, clf.predict(X_test)
    for i in range(len(y_true)):
        
        total_true.append(y_true[i])
        total_pred.append(y_pred[i])

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.553 +/- 0.003 (71471/129189)'

In [597]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

'Precision: 0.022 | Recall: 0.663 | F1: 0.043'

In [598]:
clf.best_params_

{u'C': 4.5}

<h3> Train on full talks, test on full talks </h3>
This takes a while

In [599]:
total_true = []
total_pred = []

for train_index, test_index in kf:
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    X_train, y_train = get_unbalanced_train_data_and_labels(t_train)
    X_test, y_test = get_unbalanced_train_data_and_labels(t_test)

    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)

    X_train, X_test = X_train[:,73:], X_test[:,73:]
    
    logreg=linear_model.LogisticRegression(penalty='l1')
    clf = logreg
    clf = GridSearchCV(logreg, {'C':(0.001, .01, .1, 1, 3, 5)}, cv=3)
    #clf = GridSearchCV(logreg, {'C':(3,3.5,4,4.5)}, cv=3)
    clf.fit(X_train, y_train)
    
    y_true, y_pred = y_test, clf.predict(X_test)
    for i in range(len(y_true)):      
        total_true.append(y_true[i])
        total_pred.append(y_pred[i])

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.985 +/- 0.001 (127233/129189)'

In [617]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

'Precision: 0.000 | Recall: 0.000 | F1: 0.000'

In [78]:
clf.best_params_

{u'C': 3.5}

<h2> MLP on a single sentence </h2>

In [49]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, Dropout
import keras.optimizers
from keras.models import load_model
import keras.regularizers
from keras.regularizers import l2, l1

def initialize_mlp2():
    model = Sequential()
    model.add(Dense(600, use_bias=True,input_dim=613))#1924
    model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.8))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

def initialize_mlp():
    model = Sequential()
    model.add(Dense(600, use_bias=True,input_dim=386))#1924
    #model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.8))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [93]:
m = initialize_mlp()

In [52]:
kf = KFold(len(all_transcripts), n_folds=10)

In [83]:
#t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
#X_train, y_train = get_balanced_train_data_and_labels(t_train)

In [50]:
import sklearn

In [54]:
#Keras model FF

#tf.reset_default_graph()



total_true = []
total_pred = []

for train_index, test_index in kf:  
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)
    X_test, y_test = get_balanced_train_data_and_labels_with_audio(t_test)
    
    for i in xrange(X_train.shape[1]):
        mean = np.mean(X_train[:,i])
        X_train[:,i] -= mean
        X_test[:,i] -=mean
        std = np.std(X_train[:,i])
        X_train[:,i] /= std
        X_test[:,i] /= std

    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)
    
    #X_train = sklearn.preprocessing.normalize(X_train,return_norm=True)

    X_train_words, X_test_words = X_train[:,73:373], X_test[:,73:373]
    X_train_audio, X_test_audio = X_train[:,373:], X_test[:,373:]

    best_acc = 0
    m = initialize_mlp()
    val_index = int(len(X_train)*0.9)
    val_data, val_labels = (X_train_words[val_index:], y_train[val_index:])
    train_data, train_labels = (X_train_words[0:val_index], y_train[0:val_index])

    max_epochs = 30; e = 0
    while e < max_epochs:                            
        m.fit(X_train_words[0:val_index], y_train[0:val_index], nb_epoch=1,batch_size=128,shuffle=True)
        acc = m.evaluate(val_data, val_labels)[1]
        if acc > best_acc: 
            best_acc = acc
            m.save('best_ff_model.h5')
        e+=1

    m = load_model('best_ff_model.h5')
    
    word_features_train = np.maximum(np.dot(X_train_words,m.get_weights()[0]) + m.get_weights()[1],0)
    audio_features_train = X_train_audio
    all_features_train = np.hstack([word_features_train,audio_features_train])
    
    word_features_test = np.maximum(np.dot(X_test_words,m.get_weights()[0]) + m.get_weights()[1],0)
    audio_features_test = X_test_audio
    all_features_test = np.hstack([word_features_test,audio_features_test])
    
    m2 = initialize_mlp2()
    max_epochs = 120; e = 0
    best_acc2 = 0
    while e < max_epochs:                            
        m2.fit(all_features_train[0:val_index], y_train[0:val_index], nb_epoch=1,batch_size=128,shuffle=True)
        acc2 = m2.evaluate(all_features_train[val_index:], val_labels)[1]
        if acc2 > best_acc2: 
            best_acc2 = acc2
            m2.save('best_ff_model2.h5')
        e+=1

    m2 = load_model('best_ff_model2.h5')
    
    #logreg=linear_model.LogisticRegression(penalty='l1',C=3.5)
    #clf = logreg

    #clf = GridSearchCV(logreg, {'C':(3,3.5,4,4.5)}, cv=3)
    #clf.fit(all_features_train, y_train)
    
    #y_true, y_pred = y_test, clf.predict(all_features_test)

    y_true, y_pred = y_test, np.round(m2.predict(all_features_test))
    for i in range(len(y_true)):
        total_true.append(y_true[i])
        total_pred.append(y_pred[i][0])
        
        

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

ValueError: Error when checking input: expected dense_1_input to have shape (None, 386) but got array with shape (3105, 300)

In [1489]:
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.709 +/- 0.014 (2708/3820)'

In [1490]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

'Precision: 0.715 | Recall: 0.695 | F1: 0.705'

In [357]:
X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)
X_test, y_test = get_balanced_train_data_and_labels_with_audio(t_test)

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

for i in xrange(X_train.shape[1]):
    mean = np.mean(X_train[:,i])
    X_train[:,i] -= mean
    X_test[:,i] -=mean
    std = np.std(X_train[:,i])
    X_train[:,i] /= std
    X_test[:,i] /= std

In [358]:
def initialize_mlp():
    model = Sequential()
    model.add(Dense(150, use_bias=True,input_dim=313))#1924
    model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.8))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [420]:
indices = []
for train_index, test_index in kf:
    indices.append((train_index,test_index))
#t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
#X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)
#X_test, y_test = get_balanced_train_data_and_labels_with_audio(t_test)

In [490]:
train_index, test_index = indices[1]

In [491]:
t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)
X_test, y_test = get_balanced_train_data_and_labels_with_audio(t_test)

In [492]:
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

for i in xrange(X_train.shape[1]):
    mean = np.mean(X_train[:,i])
    X_train[:,i] -= mean
    X_test[:,i] -=mean
    std = np.std(X_train[:,i])
    X_train[:,i] /= std
    X_test[:,i] /= std

In [493]:
m = initialize_mlp()

In [494]:
m.fit(X_train[:,73:],y_train,epochs=20,batch_size=128)

In [495]:
m.evaluate(X_train[:,73:],y_train,batch_size=256)

[0.69314718575691603, 0.50000000078878415]

In [496]:
m.evaluate(X_test[:,73:],y_test,batch_size=256)

[0.69314719078152676, 0.50000000970308167]

In [476]:
len(t_test)

87

In [412]:
y_true, y_pred = y_test, np.round(m.predict(X_test[:,73:]))

In [413]:
total_true = []
total_pred = []

In [414]:
for i in range(len(y_true)):
    total_true.append(y_true[i])
    total_pred.append(y_pred[i][0])
        
        

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.752 +/- 0.041 (313/416)'

In [403]:
kf = KFold(len(all_transcripts), n_folds=10)

In [1552]:
np.sum(m.predict_classes(X_train,batch_size=512) == 0)

0

In [406]:
#Keras model FF

total_true = []
total_pred = []

for train_index, test_index in kf:  
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)
    X_test, y_test = get_balanced_train_data_and_labels_with_audio(t_test)

    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)

    for i in xrange(X_train.shape[1]):
        mean = np.mean(X_train[:,i])
        X_train[:,i] -= mean
        X_test[:,i] -=mean
        std = np.std(X_train[:,i])
        X_train[:,i] /= std
        X_test[:,i] /= std
    
    m = initialize_mlp()

    m.fit(X_train[:,73:],y_train,epochs=50,batch_size=128)
    
    y_true, y_pred = y_test, np.round(m.predict(X_test[:,73:]))
    for i in range(len(y_true)):
        total_true.append(y_true[i])
        total_pred.append(y_pred[i][0])
        
        

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

'Accuracy: 0.716 +/- 0.014 (2734/3820)'

(416, 386)

In [407]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

'Precision: 0.749 | Recall: 0.648 | F1: 0.695'

In [194]:
clf.best_estimator_

LogisticRegression(C=3.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty=u'l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [161]:
X_train, y_train = get_balanced_train_data_and_labels_with_audio(t_train)

In [162]:
X_train.shape

(3440, 386)

In [176]:
X_train_words = X_train[:,73:373]
X_train_words.shape

(3440, 300)

In [184]:
word_features = np.maximum(np.dot(X_train_words,m.get_weights()[0]) + m.get_weights()[1],0)

In [185]:
audio_features = X_train[:,373:]

In [186]:
all_features = np.hstack([word_features, audio_features])

In [187]:
all_features.shape

(3440, 613)

In [175]:
m.get_weights()[2].shape

(600, 1)

<h1> Train LSTM over series of 5 Sentences </h1>

In [1852]:
list(range(5,10))

[5, 6, 7, 8, 9]

In [1853]:
def get_applause_indices(transcript):
    a = [s[2] for s in transcript.sentences]
    return list(np.nonzero(a)[0])

def get_non_applause_indices(transcript):
    return [index for index, s in enumerate(transcript.sentences) if s[2] == 0]

def extract_features_from_sentence_list(l):
    return np.array([s[1] for s in l])

def extract_applause_sequences(transcript,num_sentences):
    applause_indices = [i for i in get_applause_indices(transcript) if i > num_sentences-1]
    sentence_lists = [transcript.sentences[i-num_sentences+1:i+1] for i in applause_indices]
    feature_lists = [extract_features_from_sentence_list(s) for s in sentence_lists]
    return np.array(feature_lists)

def extract_non_applause_sequences(transcript,num_sentences,n):
    non_applause_indices = [i for i in get_non_applause_indices(transcript) if i > num_sentences-1]
    non_applause_indices = shuffle(non_applause_indices)[0:n]
    sentence_lists = [transcript.sentences[i-num_sentences+1:i+1] for i in non_applause_indices]
    return np.array([extract_features_from_sentence_list(s) for s in sentence_lists])

def extract_applause_sequences_with_audio(transcript,num_sentences):
    applause_indices = [i for i in get_applause_indices(transcript) if i > num_sentences-1]
    index_lists = [list(range(i-num_sentences+1,i+1)) for i in applause_indices]
    feature_lists = [[get_feature(transcript,i) for i in index_list] for index_list in index_lists]
    return np.array(feature_lists)

def extract_non_applause_sequences_with_audio(transcript,num_sentences,n):
    non_applause_indices = [i for i in get_non_applause_indices(transcript) if i > num_sentences-1]
    non_applause_indices = shuffle(non_applause_indices)[0:n]
    index_lists = [list(range(i-num_sentences+1,i+1)) for i in non_applause_indices]
    feature_lists = [[get_feature(transcript,i) for i in index_list] for index_list in index_lists]
    return np.array(feature_lists)

In [1854]:
def get_words_in_sentence(sentence):
    return [word for word in nlp(sentence) if re.search("\S", word.string) != None]

def count_words_in_sentence(sentence):
    return len(get_words_in_sentence(sentence))

def get_num_previous_words(transcript, index): 
    return int(np.sum([count_words_in_sentence(s) for s in [s[0] for s in transcript.sentences[0:index]]]))

def get_vector_list_from_word_list(word_list):
    return [word.vector for word in word_list]

def get_preceding_words(transcript, index, num_words):
    sentences = [s[0] for s in transcript.sentences]
    words = []
    i = index
    while len(words) < num_words:
        words = get_words_in_sentence(sentences[i]) + words
        i -= 1
    return words[len(words)-num_words:]

def get_preceding_word_vectors(transcript, index, num_words):
    vecs = []
    i = index
    while len(vecs) < num_words:
        #vecs = list(transcript.sentences[i][3]) + vecs
        vecs = list(transcript.word_features[i]) + vecs
    return (vecs)[-num_words:]
    
def extract_applause_word_sequences(transcript, num_words):
    applause_indices = [i for i in get_applause_indices(transcript) if t.count_preceding_words(i) > num_words]
    vector_lists = [get_preceding_word_vectors(transcript, i, num_words) for i in applause_indices]
    #word_lists = [get_preceding_words(transcript, i, num_words) for i in applause_indices]
    #vector_lists = [get_vector_list_from_word_list(wl) for wl in word_lists]
    return np.array(vector_lists)

def extract_non_applause_word_sequences(transcript,num_words,n):
    non_applause_indices = [i for i in get_non_applause_indices(transcript) if t.count_preceding_words(i) > num_words]
    non_applause_indices = shuffle(non_applause_indices)[0:n]
    vector_lists = [get_preceding_word_vectors(transcript, i, num_words) for i in non_applause_indices]
    #word_lists = [get_preceding_words(transcript, i, num_words) for i in non_applause_indices]
    #vector_lists = [get_vector_list_from_word_list(wl) for wl in word_lists]
    return np.array(vector_lists)

In [1855]:
n_features = 373
n_categories = 1
n_hidden=200
num_sentences = 5
number_of_layers=1
learning_rate = 0.1
batch_size = 10

<h3> Extract applause with a context of 4 previous sentences and train LSTM on balanced data </h3>

In [1856]:
def get_balanced_lstm_applause_features(all_transcripts, sequence_len, sequence_type = 'sentence'):
    applause_yes = None
    applause_no = None
    for t in all_transcripts:
        if sequence_type == 'sentence':
            positives = extract_applause_sequences_with_audio(t,sequence_len)
            negatives = extract_non_applause_sequences_with_audio(t,sequence_len,len(positives))
        elif sequence_type == 'word':
            positives = extract_applause_word_sequences(t,sequence_len)
            negatives = extract_non_applause_word_sequences(t,sequence_len,len(positives))
        if len(positives) != 0:
            if applause_yes is None:
                applause_yes = positives
                applause_no = negatives
            else:
                applause_yes = np.vstack([applause_yes, positives])
                applause_no = np.vstack([applause_no, negatives])
    return applause_yes, applause_no

def format_label(l, sequence_len):
    return np.array(list(np.zeros(sequence_len-1)) + [l]).reshape((sequence_len,1))

def train_tf(train_data, train_labels):
    for i in range(len(train_data)):
        sess.run(optimizer,feed_dict={x:train_data[i],y:train_labels[i].reshape((5,1)),seq_length:num_sentences})
        
def evaluate_on_data(test_data, test_labels):
    num_correct = 0
    for i in range(len(test_data)):
        num_correct += sess.run(accuracy,feed_dict={x:test_data[i],y:test_labels[i].reshape((5,1)),seq_length:num_sentences})
    return(num_correct / len(test_data))

def predict_on_data(test_data):
    return [np.round(sess.run(prediction,feed_dict={x:test_data[i],seq_length:num_sentences}))[0][0] for i in range(len(test_data))]

def get_balanced_lstm_features_and_labels(all_transcripts, sequence_len, sequence_type = 'sentence'):
    applause_yes, applause_no = get_balanced_lstm_applause_features(all_transcripts, sequence_len, sequence_type = sequence_type)
    X = np.vstack([applause_yes,applause_no])
    Y = np.concatenate([np.ones(len(applause_yes)),np.zeros(len(applause_no))])
    Y = np.array([format_label(l, sequence_len) for l in Y]).reshape((len(Y),-1))
    return X, Y

In [14]:
# Set up tensorflow graph

tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_categories])
seq_length = tf.placeholder(tf.int32)

weights = {'out': tf.Variable(tf.random_normal([n_hidden, n_categories])),}
biases = {'out': tf.Variable(tf.random_normal([n_categories]))}

fc1 = tf.reshape(x, [1, -1, n_features])
lstm_fw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
lstm_fw_multicell = tf.contrib.rnn.MultiRNNCell([lstm_fw_cell] * number_of_layers, state_is_tuple=True)

output_fw, state_fw = tf.nn.dynamic_rnn(lstm_fw_multicell,fc1,dtype='float32',sequence_length=tf.reshape(seq_length, [1]))
outputs_fw = tf.reshape(output_fw, [seq_length, n_hidden])
out = tf.matmul(outputs_fw, weights['out']) + biases['out']
pred = tf.gather(out,[num_sentences-1])
end_label = tf.gather(y,[num_sentences-1])
#end_label = tf.reshape(tf.gather(tf.reshape(y,[-1]),[num_sentences-1]),[1,1])

cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=end_label)
cost2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=out, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)#(learning_rate=0.1, l2_regularization_strength=0.1).minimize(cost)
#optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate, l2_regularization_strength=1.0).minimize(cost)
prediction = tf.sigmoid(pred)
correct_pred = tf.equal(tf.greater_equal(prediction,0.5), tf.equal(end_label,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [15]:
#kf = KFold(len(all_transcripts), n_folds=10)

In [29]:
total_true = []
total_pred = []

for train_index, test_index in kf:
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    
    X_train, y_train = get_balanced_lstm_features_and_labels(t_train, num_sentences=5)
    X_test, y_test = get_balanced_lstm_features_and_labels(t_test, num_sentences=5)
    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)
    #X_train, X_test = X_train[:,:,73:], X_test[:,:,73:]
    
    best_acc = 0
    best_weights = np.zeros(n_hidden)
    best_bias = 0.
    val_index = int(len(X_train)*0.9)
    val_data, val_labels = (X_train[val_index:], y_train[val_index:])
    train_data, train_labels = (X_train[0:val_index], y_train[0:val_index])
    
    max_epochs = 10
    sess.run(init)
    for i in range(max_epochs):
        train_data, train_labels = shuffle(train_data, train_labels)
        train_tf(train_data, train_labels)
        acc = evaluate_on_data(val_data, val_labels)
        if acc > best_acc: 
            best_acc = acc
            best_weights = sess.run(weights['out'])
            best_bias = sess.run(biases['out'])
        
    assign_weights_op = tf.assign(weights['out'], best_weights)
    assign_bias_op = tf.assign(biases['out'], best_bias)
    sess.run(assign_weights_op)
    y_true, y_pred = y_test[:,num_sentences-1], predict_on_data(X_test)
    
    for i in range(len(y_true)):      
        total_true.append(y_true[i])
        total_pred.append(y_pred[i])

total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

TypeError: get_balanced_lstm_features_and_labels() got an unexpected keyword argument 'num_sentences'

In [ ]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

<h2> Implement LSTM again in Keras to compare </h2>

In [1857]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, Dropout
import keras.optimizers
from keras.models import load_model
import keras.regularizers
from keras.regularizers import l2, l1

In [2262]:
from keras.layers import LSTM
from keras.layers import regularizers
from keras.layers import Bidirectional

def initialize_lstm_model():
    model = Sequential()
    model.add(Dropout(0.5,input_shape=(2,313)))
    #model.add(Conv1D(6,3,padding='same',input_shape=(5, 386)))
    #model.add(keras.layers.BatchNormalization())
    #model.add(Activation('relu'))
    #model.add(MaxPooling1D(pool_size=3))
    #model.add(Conv1D(36,3,padding='same'))
    #model.add(keras.layers.BatchNormalization())
    #model.add(Activation('relu'))
    #model.add(MaxPooling1D(pool_size=3))
    #model.add(Flatten())
    #model.add(Dense(50))
    #model.add(Dropout(0.5))
    model.add(LSTM(100,input_shape=(None,313),return_sequences=False,dropout=0.8))
    #model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.8),input_shape=(None, 313)))
    #model.add(Bidirectional(LSTM(100, return_sequences=False, dropout=0.5),input_shape=(None, 313)))
    #model.add(Dropout(0.5))
    #model.add(Activation("relu"))
    #model.add(LSTM(50,return_sequences=False))
    #model.add(Dropout(0.5))
    #model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [2263]:
m = initialize_lstm_model()

In [2264]:
#train_data_2 = np.array([t [3:,:] for t in train_data])
#test_data_2 = np.array([t [3:,:] for t in test_data])
#train_data_2.shape

In [2265]:
len(all_transcripts)*0.8

692.8000000000001

In [2266]:
t_train = all_transcripts[0:693]
t_test = all_transcripts[693:]

In [2267]:
X_train, y_train = get_balanced_lstm_features_and_labels(t_train, sequence_len=2, sequence_type='sentence')
X_test, y_test = get_balanced_lstm_features_and_labels(t_test, sequence_len=2, sequence_type='sentence')
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [2268]:
X_train.shape

(2982, 2, 386)

In [2269]:
s = X_train.shape
s2 = X_test.shape
temp_x_train = X_train.reshape((s[0]*s[1],s[2]))
temp_x_test = X_test.reshape((s2[0]*s2[1],s2[2]))
for i in xrange(temp_x_train.shape[1]):
    mean = np.mean(temp_x_train[:,i])
    temp_x_train[:,i] -= mean
    temp_x_test[:,i] -=mean
    std = np.std(temp_x_train[:,i])
    temp_x_train[:,i] /= std
    temp_x_test[:,i] /= std

In [2270]:
X_train = temp_x_train.reshape(s)
X_test = temp_x_test.reshape(s2)

In [2271]:
X_train = X_train[:,:,73:]
X_train.shape

(2982, 2, 313)

In [2272]:
X_test = X_test[:,:,73:]

In [2273]:
m = initialize_lstm_model()

In [2349]:
m.fit(X_train, y_train[:,-1], nb_epoch=1, batch_size=128,shuffle=True)

In [2350]:
m.evaluate(X_train,y_train[:,-1],batch_size=512)

[0.32114363478223085, 0.88598256035989598]

In [2351]:
m.evaluate(X_test,y_test[:,-1],batch_size=512)

[0.62136490557128021, 0.67042604335268641]

In [18]:
kf = KFold(len(all_transcripts), n_folds=10)

In [ ]:
#Keras model

#tf.reset_default_graph()


total_true = []
total_pred = []

for train_index, test_index in kf:  
    t_train, t_test = np.array(all_transcripts)[train_index], np.array(all_transcripts)[test_index]
    
    X_train, y_train = get_balanced_lstm_features_and_labels(t_train, sequence_len=36, sequence_type='word')
    X_test, y_test = get_balanced_lstm_features_and_labels(t_test, sequence_len=36, sequence_type='word')
    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)
    
    # Don't include labels for the whole sequence as we're only predicting at the end of the sequence
    y_train = np.array([l[-1] for l in y_train])
    y_test = np.array([l[-1] for l in y_test])
    
    best_acc = 0
    m = initialize_lstm_model()
    val_index = int(len(X_train)*0.9)
    val_data, val_labels = (X_train[val_index:], y_train[val_index:])
    train_data, train_labels = (X_train[0:val_index], y_train[0:val_index])
    
    max_epochs = 1; e = 0
    while e < max_epochs:                            
        m.fit(X_train[0:val_index], y_train[0:val_index], nb_epoch=1,batch_size=128,shuffle=True)
        acc = m.evaluate(val_data, val_labels)[1]
        if acc > best_acc: 
            best_acc = acc
            m.save('best_model.h5')
        e+=1
        
    m = load_model('best_model.h5')
    
    y_true, y_pred = y_test, np.round(m.predict(X_test))
    for i in range(len(y_true)):
        total_true.append(y_true[i])
        total_pred.append(y_pred[i][0])
    
total_correct = np.sum(np.array(total_true) == np.array(total_pred))
acc=float(total_correct) / len(total_true)
std=math.sqrt( (acc * (1-acc)) / len(total_true) )
str("Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true)))

In [ ]:
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
str("Precision: %.3f | Recall: %.3f | F1: %.3f" % (precision, recall, f1))

In [ ]:
acc